In [1]:
import time
import numpy as np

import hoomd
import gsd.hoomd

from type_updater import TypeUpdater

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=59920)
sim.create_state_from_gsd(filename='./DATA/compress.gsd', frame=-1)

#### integrator

In [3]:
def compute_d1d2(alpha):
        
    const = (4*np.pi/3)**(-1/3)
    volume_of_two_types = 2.0
    pre_factor = const * (volume_of_two_types)**(1/3)

    r1 = pre_factor * (1/(1+alpha**3))**(1/3)
    r2 = alpha * r1

    return 2*r1, 2*r2

alpha = 0.42
d1, d2 = compute_d1d2(alpha)

mc = hoomd.hpmc.integrate.Sphere(default_d=0.3, default_a=0.4)
mc.shape["A"] = dict(diameter=d1)
mc.shape["B"] = dict(diameter=d2)

#### movesize tuner

In [4]:
tune = hoomd.hpmc.tune.MoveSize.scale_solver(
    moves=['a', 'd'],
    target=0.2,
    trigger=hoomd.trigger.And([
        hoomd.trigger.Periodic(100),
        hoomd.trigger.Before(sim.timestep + 5000)
    ]))

#### type updater

In [5]:
rng = np.random.default_rng(9920)
typeupdater = hoomd.update.CustomUpdater(action=TypeUpdater(rng=rng), trigger=1)

#### write

In [6]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

gsd_writer = hoomd.write.GSD(filename='./DATA/equilibriate.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All(),
                             dynamic=['attribute'],
                             log=logger)

/Users/lina492375qw1188/opt/miniconda3/envs/patchy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x12464cd30> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


#### attaching operations

In [7]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc
sim.operations.tuners.append(tune)
sim.operations.updaters.append(typeupdater)

#### tuning run

In [8]:
sim.run(5000)

#### check acceptance ratio

In [9]:
sim.run(100)

translate_moves = mc.translate_moves
mc.translate_moves[0] / sum(mc.translate_moves)

0.205615

#### run simulation

In [10]:
start = time.time()

sim.run(1e4)

print('Time elapsed', time.time()-start)

Time elapsed 52.52597498893738


In [11]:
typeupdater.moves

(5, 15095)